In [1]:
%matplotlib inline
import os
import nltk
import re
import numpy as np
import regex
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
import pandas as pd
import string

In [2]:
loose=pd.read_csv('original-sheets\Responses-Loose.csv')
student_info=pd.read_csv('corpus-files\student_information.csv')
course=pd.read_csv('corpus-files\course.csv')
answer_df=pd.read_csv('answer.csv',index_col = 'answer_id', low_memory=False)


In [3]:
#dropping to only relevant columns
loose=loose.drop(columns=['3Langs','Item','gender','course_id','class_id','version','text1_len','text2_len','text1',
                         'text2 (line breaks/extra spaces removed, spaces added to reach 60)','Judgement','Notes'])

In [4]:
course_=course.drop(columns=['class_id','semester','section'])

student=student_info.drop(columns=['birth_year','language_used_at_home','non_native_language_1','yrs_of_study_lang1',
                                  'study_in_classroom_lang1','ways_of_study_lang1','non_native_language_2','study_in_classroom_lang2',
                                  'ways_of_study_lang2','non_native_language_3','yrs_of_study_lang3','study_in_classroom_lang3',
                                  'ways_of_study_lang3','yrs_of_english_learning','yrs_in_english_environment','yrs_of_study_lang2'])
answer_=answer_df[['question_id','anon_id','course_id','version','text_len','text','tokens']].reset_index()

In [5]:
#This function removes all of the rows that contain a string in the passed column
def rmStr (df, col):
    row_index=0
    row_ind=[]
    for i in df[col]:
        try:
            int(i)
        except:
            row_ind.append(row_index)
        row_index+=1
    return df.drop(labels=row_ind,axis=0)

In [6]:
#Cleaning the answer df
col='course_id'
answer=rmStr(answer_, col) #removes strings from answer_.course_id 

In [7]:
# Merging the datasets

#merge_ans=answer_[['anon_id','course_id']] #sets answer df up for merging 
#student_merge=student.drop(columns=['course_history']) #set student df up for merging

student_ans=student.merge(answer_,on='anon_id').astype({'course_id':'int64'}) #merges student and answers

stu_ans_crs=student_ans.merge(course_, on='course_id')
#.drop(columns=['age','course_id','version','tokens']) #merges the student-answers df with course df

In [8]:
# function that finds the unique values in a list
def get_uniques(x):
    return list(x.unique())

# Whole Dataset

In [9]:
stu_ans_crs

,anon_id,gender,native_language,course_history,age,answer_id,question_id,course_id,version,text_len,text,tokens,level_id
0,do6,Female,Russian,5;11;18;24;29;105;117;153,31.0,150,4,117,1,299,Some people prefer eat out and some like doing...,"['Some', 'people', 'prefer', 'eat', 'out', 'an...",5
1,do6,Female,Russian,5;11;18;24;29;105;117;153,31.0,1221,97,117,1,288,My opinion is that a person does need educatio...,"['My', 'opinion', 'is', 'that', 'a', 'person',...",5
2,do6,Female,Russian,5;11;18;24;29;105;117;153,31.0,1957,189,117,1,321,There are two national rooms in the Cathedral ...,"['There', 'are', 'two', 'national', 'rooms', '...",5
3,do6,Female,Russian,5;11;18;24;29;105;117;153,31.0,2164,190,117,1,464,There are two nation rooms in the Cathedral of...,"['There', 'are', 'two', 'nation', 'rooms', 'in...",5
4,bv5,Male,Arabic,3;9;15;21;27;49;57;71;82;93;117,21.0,151,4,117,1,315,"""Not all learning takes place in the classroom...","['``', 'Not', 'all', 'learning', 'takes', 'pla...",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
46225,cy7,Female,Korean,NaN,NaN,47682,6066,1034,1,10,1. emphasis\n2. appropriate\n3. requires\n4. a...,"['1', '.', 'emphasis', '2', '.', 'appropriate'...",3
46226,fp7,Female,Turkish,NaN,NaN,47823,6066,1034,1,10,1.emphasis \n2.appropriate\n3.requires\n4.anal...,"['1', '.', 'emphasis', '2', '.', 'appropriate'...",3
46227,fq6,Male,Chinese,NaN,NaN,47824,6066,1034,1,10,1.emphasis\n2.appropriate\n3.requires\n4.analy...,"['1', '.', 'emphasis', '2', '.', 'appropriate'...",3
46228,di6,Male,Chinese,NaN,NaN,47787,6066,1034,1,10,1 emphasis\n2 normal\n3 requires\n4 analyze\n5...,"['1', 'emphasis', '2', 'normal', '3', 'require...",3


In [10]:
sac_lvls=stu_ans_crs[['anon_id','native_language','level_id','question_id','text']]
sac_lvls_rkc=sac_lvls

index=0
for i in sac_lvls_rkc.native_language:
    if (i != 'Korean' and i != 'Spanish' and i != 'Chinese'):
        sac_lvls_rkc=sac_lvls_rkc.drop(index)
    index+=1

sac_lvls_rkc=sac_lvls_rkc.reset_index()

sac_lvls_rkc

,index,anon_id,native_language,level_id,question_id,text
0,13,ax4,Chinese,5,48,The Effects on Students Who Have a Part-time J...
1,14,ax4,Chinese,5,97,There are many different kinds of people in th...
2,15,ax4,Chinese,5,74,Should the Death Penalty Exist\nThe death pena...
3,16,ax4,Chinese,5,119,Should the Death Penalty Exist?\nThe death pen...
4,17,ax4,Chinese,5,189,The Nationality rooms built in the Cathedral o...
...,...,...,...,...,...,...
19633,46218,cz3,Korean,3,6087,I have studied Visual Design since 2002. I gra...
19634,46219,cz3,Korean,3,6119,I will describe about my ideal home. I want qu...
19635,46225,cy7,Korean,3,6066,1. emphasis\n2. appropriate\n3. requires\n4. a...
19636,46227,fq6,Chinese,3,6066,1.emphasis\n2.appropriate\n3.requires\n4.analy...


In [11]:
# allows us to see what levels each anon_id participated in 
lvl_list=sac_lvls_rkc.groupby('anon_id').agg(n_uniq=('question_id','nunique'), lvl_nums=('question_id',get_uniques))
lvl_list=lvl_list.reset_index()

In [12]:
unique_q=sac_lvls_rkc.groupby('native_language').agg(n_uniq=('question_id','nunique'))
unique_q=unique_q.reset_index()

In [13]:
unique_q

,native_language,n_uniq
0,Chinese,3708
1,Korean,3345
2,Spanish,1513


In [14]:
unique_t=sac_lvls_rkc.groupby('native_language').agg(n_uniq=('text','nunique'))
unique_t=unique_t.reset_index()
unique_t

,native_language,n_uniq
0,Chinese,7652
1,Korean,7915
2,Spanish,1794


In [15]:
unique_a=sac_lvls_rkc.groupby('native_language').agg(n_uniq=('anon_id','nunique'))
unique_a=unique_a.reset_index()
unique_a

,native_language,n_uniq
0,Chinese,220
1,Korean,214
2,Spanish,57


In [16]:
sac_lvls_rkc=sac_lvls_rkc.reset_index()
sac_lvls_rkc

,level_0,index,anon_id,native_language,level_id,question_id,text
0,0,13,ax4,Chinese,5,48,The Effects on Students Who Have a Part-time J...
1,1,14,ax4,Chinese,5,97,There are many different kinds of people in th...
2,2,15,ax4,Chinese,5,74,Should the Death Penalty Exist\nThe death pena...
3,3,16,ax4,Chinese,5,119,Should the Death Penalty Exist?\nThe death pen...
4,4,17,ax4,Chinese,5,189,The Nationality rooms built in the Cathedral o...
...,...,...,...,...,...,...,...
19633,19633,46218,cz3,Korean,3,6087,I have studied Visual Design since 2002. I gra...
19634,19634,46219,cz3,Korean,3,6119,I will describe about my ideal home. I want qu...
19635,19635,46225,cy7,Korean,3,6066,1. emphasis\n2. appropriate\n3. requires\n4. a...
19636,19636,46227,fq6,Chinese,3,6066,1.emphasis\n2.appropriate\n3.requires\n4.analy...


In [17]:
#dividing my level 3 and language
sac_lvls_rkc3=sac_lvls_rkc

index=0
for i in sac_lvls_rkc3.level_id:
    if i!=3:
        sac_lvls_rkc3=sac_lvls_rkc3.drop(index)
    index+=1
    
sac_lvls_rkc3

,level_0,index,anon_id,native_language,level_id,question_id,text
1311,1311,2478,fk5,Korean,3,34,I favorite subway restaurant and outback staek...
1312,1312,2479,fk5,Korean,3,31,Here are a few instructions for typing in Engl...
1313,1313,2480,fk5,Korean,3,98,"Personal Letter\n \n October 16, 2006 \n\nDear..."
1314,1314,2481,fk5,Korean,3,98,"Personal Letter\n \n October 20, 2006 \nDear B..."
1315,1315,2482,fk5,Korean,3,121,"Personal Letter\n \n October 20, 2006 \nDear B..."
...,...,...,...,...,...,...,...
19633,19633,46218,cz3,Korean,3,6087,I have studied Visual Design since 2002. I gra...
19634,19634,46219,cz3,Korean,3,6119,I will describe about my ideal home. I want qu...
19635,19635,46225,cy7,Korean,3,6066,1. emphasis\n2. appropriate\n3. requires\n4. a...
19636,19636,46227,fq6,Chinese,3,6066,1.emphasis\n2.appropriate\n3.requires\n4.analy...


In [18]:
unique_q=sac_lvls_rkc3.groupby('native_language').agg(n_uniq=('question_id','nunique'))
unique_q=unique_q.reset_index()

unique_t=sac_lvls_rkc3.groupby('native_language').agg(n_uniq=('text','nunique'))
unique_t=unique_t.reset_index()

unique_a=sac_lvls_rkc3.groupby('native_language').agg(n_uniq=('anon_id','nunique'))
unique_a=unique_a.reset_index()

In [19]:
unique_q

,native_language,n_uniq
0,Chinese,880
1,Korean,669
2,Spanish,302


In [20]:
unique_t

,native_language,n_uniq
0,Chinese,1777
1,Korean,1466
2,Spanish,397


In [21]:
unique_a

,native_language,n_uniq
0,Chinese,86
1,Korean,88
2,Spanish,18


In [22]:
#dividing by level 4 and language
sac_lvls_rkc4=sac_lvls_rkc

index=0
for i in sac_lvls_rkc4.level_id:
    if i!=4:
        sac_lvls_rkc4=sac_lvls_rkc4.drop(index)
    index+=1
    
sac_lvls_rkc4

,level_0,index,anon_id,native_language,level_id,question_id,text
72,72,189,cd6,Chinese,4,116,1)Creating mind movies.\n2)Setting the right g...
73,73,190,cd6,Chinese,4,156,reserve (adj.)\nDefinition: an amount of somet...
74,74,191,gy3,Chinese,4,44,vote \npersevere\n\nprohibit
75,75,192,gy3,Chinese,4,44,vote: verb\n We will vote our class monitor. \...
76,76,195,gq4,Spanish,4,44,"1. interpret:\nI went to the museum yerterday,..."
...,...,...,...,...,...,...,...
19400,19400,45191,ff6,Chinese,4,6030,"If I could choose, I would not like to live in..."
19401,19401,45192,ff6,Chinese,4,6117,I consider that the most effective way to mark...
19402,19402,45197,ev5,Chinese,4,6002,Hikikomori lifestyle is the most unusual of al...
19403,19403,45198,ev5,Chinese,4,6030,"I don't like to live in the leisureville, beca..."


In [23]:
unique_q=sac_lvls_rkc4.groupby('native_language').agg(n_uniq=('question_id','nunique'))
unique_q=unique_q.reset_index()

unique_t=sac_lvls_rkc4.groupby('native_language').agg(n_uniq=('text','nunique'))
unique_t=unique_t.reset_index()

unique_a=sac_lvls_rkc4.groupby('native_language').agg(n_uniq=('anon_id','nunique'))
unique_a=unique_a.reset_index()

In [24]:
unique_t

,native_language,n_uniq
0,Chinese,2712
1,Korean,2955
2,Spanish,697


In [25]:
unique_a

,native_language,n_uniq
0,Chinese,154
1,Korean,151
2,Spanish,40


In [26]:
#dividing my level 5 and language
sac_lvls_rkc5=sac_lvls_rkc

index=0
for i in sac_lvls_rkc5.level_id:
    if i!=5:
        sac_lvls_rkc5=sac_lvls_rkc5.drop(index)
    index+=1
    
sac_lvls_rkc5

,level_0,index,anon_id,native_language,level_id,question_id,text
0,0,13,ax4,Chinese,5,48,The Effects on Students Who Have a Part-time J...
1,1,14,ax4,Chinese,5,97,There are many different kinds of people in th...
2,2,15,ax4,Chinese,5,74,Should the Death Penalty Exist\nThe death pena...
3,3,16,ax4,Chinese,5,119,Should the Death Penalty Exist?\nThe death pen...
4,4,17,ax4,Chinese,5,189,The Nationality rooms built in the Cathedral o...
...,...,...,...,...,...,...,...
19261,19261,44724,bf7,Chinese,5,6136,"Recently, I have watched a comedy starring Mar..."
19262,19262,44751,go7,Chinese,5,6000,I am confused by the East Island status. It i...
19263,19263,44752,go7,Chinese,5,6043,"When I was in class at lunch time today, I hav..."
19264,19264,44753,go7,Chinese,5,6104,Today I have listened a part of ANON_NAME_0 S...


In [27]:
unique_q=sac_lvls_rkc5.groupby('native_language').agg(n_uniq=('question_id','nunique'))
unique_q=unique_q.reset_index()

unique_t=sac_lvls_rkc5.groupby('native_language').agg(n_uniq=('text','nunique'))
unique_t=unique_t.reset_index()

unique_a=sac_lvls_rkc5.groupby('native_language').agg(n_uniq=('anon_id','nunique'))
unique_a=unique_a.reset_index()

In [28]:
unique_t

,native_language,n_uniq
0,Chinese,3038
1,Korean,3182
2,Spanish,615


In [29]:
unique_a

,native_language,n_uniq
0,Chinese,96
1,Korean,112
2,Spanish,25


# Finding texts across all levels Whole

In [44]:
ind=0
sac_lvls_rkc=sac_lvls_rkc.drop(columns=['level_0'])
sac_lvls_rkc=sac_lvls_rkc.reset_index()
for i in sac_lvls_rkc.anon_id:
    if sac_lvls_rkc.at[ind,'level_id']==2:
        sac_lvls_rkc=sac_lvls_rkc.drop(ind)
    ind+=1
sac_lvls_rkc=sac_lvls_rkc.drop(columns=['level_0'])
sac_lvls_rkc=sac_lvls_rkc.reset_index()
sac_lvls_rkc

,level_0,index,anon_id,native_language,level_id,question_id,text
0,0,13,ax4,Chinese,5,48,The Effects on Students Who Have a Part-time J...
1,1,14,ax4,Chinese,5,97,There are many different kinds of people in th...
2,2,15,ax4,Chinese,5,74,Should the Death Penalty Exist\nThe death pena...
3,3,16,ax4,Chinese,5,119,Should the Death Penalty Exist?\nThe death pen...
4,4,17,ax4,Chinese,5,189,The Nationality rooms built in the Cathedral o...
...,...,...,...,...,...,...,...
19029,19029,46218,cz3,Korean,3,6087,I have studied Visual Design since 2002. I gra...
19030,19030,46219,cz3,Korean,3,6119,I will describe about my ideal home. I want qu...
19031,19031,46225,cy7,Korean,3,6066,1. emphasis\n2. appropriate\n3. requires\n4. a...
19032,19032,46227,fq6,Chinese,3,6066,1.emphasis\n2.appropriate\n3.requires\n4.analy...


In [45]:
ind

19034

In [46]:
 # this code gets how many times an id appeared (n_anon_ids) as well as an array of what levels they participated in (lvl_nums)
slr=sac_lvls_rkc.groupby('anon_id').agg(
    n_anon_ids_whole=('level_id', 'nunique'),
    lvl_nums_whole=('level_id', get_uniques)
)

#changes the clean_ids_lvl's index so we can access the anon_id easier
ind_whole= slr.reset_index()
ind_whole

,anon_id,n_anon_ids_whole,lvl_nums_whole
0,aa0,1,[5]
1,aa1,2,"[4, 3]"
2,aa3,1,[4]
3,aa5,1,[4]
4,aa8,2,"[5, 4]"
...,...,...,...
483,ha0,1,[5]
484,ha2,2,"[5, 4]"
485,ha6,2,"[3, 4]"
486,hb4,3,"[5, 4, 3]"


In [47]:
# for loop that checks if lvl_nums has all three levels in it (3,4, and 5)
# if there are three values in lvl_nums, the anon_id is appended to the all_three list
ind=0
wall_three_loose=[]
append=''
for i in ind_whole.anon_id:
    if ind_whole.iat[ind, 1]==3:
        append=ind_whole.at[ind,'anon_id']
        wall_three_loose.append(append)
    ind=ind+1

wall_three_loose.sort()
len(wall_three_loose)

47

In [48]:
# creates list of participants in loose sheet that are in level 3 and 4
ind=0
in34_whole=[]
append=''
for i in ind_whole.anon_id:
    if ind_whole.iat[ind, 1]==2:
        if (3 in ind_whole.iat[ind,2] and 4 in ind_whole.iat[ind,2]):
            append=ind_whole.at[ind,'anon_id']
            in34_whole.append(append)
    elif ind_whole.iat[ind,1]==3:
        append=ind_whole.at[ind,'anon_id']
        in34_whole.append(append)
    ind=ind+1
in34_whole.sort()
print(len(in34_whole))

121


In [49]:
# creates list of participants in loose sheet that are in level 3 and 4
ind=0
in45_whole=[]
append=''
for i in ind_whole.anon_id:
    if ind_whole.iat[ind, 1]==2:
        if (4 in ind_whole.iat[ind,2] and 5 in ind_whole.iat[ind,2]):
            append=ind_whole.at[ind,'anon_id']
            in45_whole.append(append)
    elif ind_whole.iat[ind,1]==3:
        append=ind_whole.at[ind,'anon_id']
        in45_whole.append(append)
    ind=ind+1
in45_whole.sort()
print(len(in45_whole))

160


In [50]:
wall_three=sac_lvls_rkc
index=0
for i in wall_three.anon_id:
    if i not in wall_three_loose:
        wall_three=wall_three.drop(index)
    index+=1
    
#wall_three=wall_three.reset_index()
wall_three

,level_0,index,anon_id,native_language,level_id,question_id,text
150,150,333,ev9,Chinese,4,2220,Dr. Schwartz's advices are very crucial and u...
151,151,334,ev9,Chinese,4,2282,There are two reasons that I think Martin is m...
152,152,335,ev9,Chinese,4,2493,"If I am going to start a new business, I will..."
155,155,338,co5,Korean,4,2220,She suggested several tips of online dating. ...
156,156,339,co5,Korean,4,2282,I am convinced that Martin really has psychic...
...,...,...,...,...,...,...,...
18635,18635,44651,bf7,Chinese,5,6040,"1. In Taiwan's middle school system, there are..."
18636,18636,44652,bf7,Chinese,5,6081,Indonesia is being threatened by tobacco indus...
18655,18655,44722,bf7,Chinese,5,6000,I am fascinated by mysteries. Mysteries are in...
18656,18656,44723,bf7,Chinese,5,6043,"No matter my present or my future life, there ..."


In [51]:
whole34=sac_lvls_rkc
index=0
for i in whole34.anon_id:
    if i not in in34_whole:
        whole34=whole34.drop(index)
    index+=1

whole34

,level_0,index,anon_id,native_language,level_id,question_id,text
74,74,191,gy3,Chinese,4,44,vote \npersevere\n\nprohibit
75,75,192,gy3,Chinese,4,44,vote: verb\n We will vote our class monitor. \...
147,147,330,ca6,Korean,4,2220,I really impressed when I read Dr.Schwartz's ...
148,148,331,ca6,Korean,4,2282,"In my opinion, I don't believe unexplainable m..."
149,149,332,ca6,Korean,4,2493,"If I'm going to start a business, I'll use th..."
...,...,...,...,...,...,...,...
18801,18801,45234,ff6,Chinese,3,5763,A day on a beach\nSee hear smell taste feel to...
18802,18802,45235,ff6,Chinese,3,5763,A day at the beach\nIt was a sunny day. We wen...
18803,18803,45238,dx1,Chinese,3,5763,See: seagoal ; wave; stones; tree ;relax peopl...
18804,18804,45239,dx1,Chinese,3,5763,I always went to the beach when I had bad emo...


In [52]:
whole45=sac_lvls_rkc
index=0
for i in whole45.anon_id:
    if i not in in45_whole:
        whole45=whole45.drop(index)
    index+=1
    
whole45

,level_0,index,anon_id,native_language,level_id,question_id,text
6,6,19,fg8,Chinese,5,4,"In Taiwan, we have a proverb, ""Far relative ca..."
7,7,20,fg8,Chinese,5,48,The Effects of Watching movies\n\nWith the dev...
8,8,21,fg8,Chinese,5,74,Making Friends on Internet\n\nWith the develop...
9,9,22,fg8,Chinese,5,97,Many people like to ask people who have good e...
10,10,23,fg8,Chinese,5,119,Making Friends on Internet\n\nWith the develop...
...,...,...,...,...,...,...,...
18753,18753,45073,bv8,Chinese,4,5912,My birthday party always cheers me up and it i...
18754,18754,45074,bv8,Chinese,4,5960,"My name is Tike, but my friends call me Tikki...."
18755,18755,45075,bv8,Chinese,4,6029,"In the whole of my life, I had experienced tha..."
18756,18756,45076,bv8,Chinese,4,6078,"2. I lived with my parents and my sibling, who..."


In [53]:
unique_t=wall_three.groupby(['native_language','level_id']).agg(n_uniq=('text','nunique'))
unique_t=unique_t.reset_index()
unique_t

,native_language,level_id,n_uniq
0,Chinese,3,287
1,Chinese,4,279
2,Chinese,5,472
3,Korean,3,358
4,Korean,4,607
5,Korean,5,810
6,Spanish,3,147
7,Spanish,4,89
8,Spanish,5,50


# Loose Dataset

In [54]:
loose=loose.dropna()
loose

,answer_id,anon_id,L1,level_id,question_id,text3_len,text3 (edits made to fix word counts)
0,141.0,aj8,Chinese,5.0,17.0,114.0,"Today, I am going to describe one of my classm..."
1,143.0,az8,Chinese,5.0,17.0,95.0,My niece is 3 years old who is my younger brot...
2,133.0,az2,Korean,5.0,17.0,128.0,"When I was in Germany, I met a friend who was ..."
3,135.0,at8,Korean,5.0,17.0,105.0,my friend is ANON_NAME_0. she is a my ELI frie...
4,188.0,eh9,Korean,5.0,17.0,97.0,Younghun is my best friend. His facial appeara...
...,...,...,...,...,...,...,...
1805,47845.0,ec1,Korean,4.0,6070.0,138.0,My best friend is Sohyun who graduated from th...
1806,47882.0,cq1,Spanish,4.0,6070.0,80.0,"My soul mate is Melissa, who has been my frien..."
1807,48246.0,bv8,Chinese,4.0,6105.0,131.0,The quote that talks about a true friend is so...
1808,48254.0,dr8,Korean,4.0,6105.0,162.0,I think this means that home is not just a pla...


In [55]:
    # this code gets how many times an id appeared (n_anon_ids) as well as an array of what levels they participated in (lvl_nums)
clean_ids_lvl_loose=loose.groupby('anon_id').agg(
    n_anon_ids_loose=('level_id', 'nunique'),
    lvl_nums_loose=('level_id', get_uniques)
)

#changes the clean_ids_lvl's index so we can access the anon_id easier
ind_loose= clean_ids_lvl_loose.reset_index()
ind_loose

,anon_id,n_anon_ids_loose,lvl_nums_loose
0,aa0,1,[5.0]
1,aa3,1,[4.0]
2,aa8,2,"[4.0, 5.0]"
3,aa9,2,"[3.0, 4.0]"
4,ab6,1,[4.0]
...,...,...,...
278,gz2,2,"[4.0, 5.0]"
279,gz5,1,[4.0]
280,gz7,1,[4.0]
281,ha0,1,[5.0]


In [56]:
# for loop that checks if lvl_nums has all three levels in it (3,4, and 5)
# if there are three values in lvl_nums, the anon_id is appended to the all_three list
ind=0
all_three_loose=[]
append=''
for i in ind_loose.anon_id:
    if ind_loose.iat[ind, 1]==3:
        append=ind_loose.at[ind,'anon_id']
        all_three_loose.append(append)
    ind=ind+1

all_three_loose.sort()
len(all_three_loose)

8

In [57]:
# creates list of participants in loose sheet that are in level 3 and 4
ind=0
in34_loose=[]
append=''
for i in ind_loose.anon_id:
    if ind_loose.iat[ind, 1]==2:
        if (3 in ind_loose.iat[ind,2] and 4 in ind_loose.iat[ind,2]):
            append=ind_loose.at[ind,'anon_id']
            in34_loose.append(append)
    elif ind_loose.iat[ind,1]==3:
        append=ind_loose.at[ind,'anon_id']
        in34_loose.append(append)
    ind=ind+1
in34_loose.sort()
print(len(in34_loose))

28


In [58]:
# creates list of participants in loose sheet that are in level 4 and 5
ind=0
in45_loose=[]
append=''
for i in ind_loose.anon_id:
    if ind_loose.iat[ind, 1]==2:
        if (4 in ind_loose.iat[ind,2] and 5 in ind_loose.iat[ind,2]):
            append=ind_loose.at[ind,'anon_id']
            in45_loose.append(append)
    elif ind_loose.iat[ind,1]==3:
        append=ind_loose.at[ind,'anon_id']
        in45_loose.append(append)
    ind=ind+1
in45_loose.sort()
print(len(in45_loose))

61


# Finding number of texts breakdown

In [59]:
index=0
textcount_loose=loose.copy()

textcount_loose['processed_text']=pd.NaT
textcount_loose['wordtype_len']=pd.NaT
words=[]
for text in textcount_loose['text3 (edits made to fix word counts)']:
    uwords=[]
    text=str(text).replace('\n',' ')
    nopunc=text.translate(str.maketrans('','',string.punctuation))
    nopunc=nopunc.lower()
    textcount_loose.at[index,'processed_text']=nopunc
    words=nopunc.strip().split(" ")
    for word in words:
        if word=='':
            words.remove(word)
        elif word not in uwords:
            uwords.append(word)
    textcount_loose.at[index,'text3_len']=len(words)
    textcount_loose.at[index,'wordtype_len']=len(uwords)
    index+=1

In [60]:
all_three_loose

['an5', 'ar8', 'bl4', 'bp4', 'bv9', 'bz2', 'fa2', 'fy1']

In [61]:
all_three=loose
index=0
for i in all_three.anon_id:
    if i not in all_three_loose:
        all_three=all_three.drop(index)
    index+=1
    
all_three=all_three.reset_index()
all_three

,index,answer_id,anon_id,L1,level_id,question_id,text3_len,text3 (edits made to fix word counts)
0,245,6225.0,fa2,Spanish,3.0,975.0,65.0,My favorite restaurant is Atria's; I like it b...
1,343,7922.0,fa2,Spanish,3.0,1177.0,84.0,There is some advice you can do to improve you...
2,390,9275.0,fa2,Spanish,3.0,1354.0,79.0,Frida Kaloo was a famous Mexican artist. She w...
3,395,9358.0,fa2,Spanish,3.0,1381.0,106.0,Here are some tips that you have to do when yo...
4,425,10869.0,fa2,Spanish,3.0,1519.0,117.0,"I miss my dogs. I have seven dogs in Paraguay,..."
...,...,...,...,...,...,...,...,...
78,1667,37576.0,bv9,Korean,4.0,4867.0,84.0,"This article is about Anita Roddick, who overc..."
79,1704,39398.0,bp4,Chinese,4.0,5044.0,94.0,"My new room is bright, warm and fragrant. This..."
80,1724,40289.0,bp4,Chinese,4.0,5125.0,147.0,A best friend is who might be another you. May...
81,1741,42090.0,bv9,Korean,5.0,5366.0,213.0,"In the afternoon, there are a car accident and..."


In [62]:
loose34=loose
index=0
for i in loose34.anon_id:
    if i not in in34_loose:
        loose34=loose34.drop(index)
    index+=1
    
loose34=loose34.reset_index()
loose34

,index,answer_id,anon_id,L1,level_id,question_id,text3_len,text3 (edits made to fix word counts)
0,64,2593.0,gv1,Korean,3.0,302.0,72.0,"While I am going to home, I listened music. Wh..."
1,65,2600.0,aa9,Korean,3.0,302.0,60.0,"While I was taking the trolley yesterday, I sa..."
2,66,2697.0,fv7,Korean,3.0,302.0,108.0,I had a reunion with my friens. They were taki...
3,73,3394.0,aa9,Korean,3.0,312.0,150.0,I do a few things before I start to write. For...
4,77,2808.0,gv1,Korean,3.0,312.0,88.0,I think thay my writing habit is very commom. ...
...,...,...,...,...,...,...,...,...
249,1747,43471.0,cs5,Chinese,3.0,5533.0,217.0,What is friend in you pinion? The friend in my...
250,1750,43550.0,cf6,Korean,3.0,5533.0,82.0,There are many reasons to travel. First a fami...
251,1755,44050.0,bp4,Chinese,5.0,5612.0,274.0,Some people think Friday and Sunday is same fo...
252,1761,45235.0,cf6,Korean,4.0,5760.0,63.0,I think I could not do the samething as articl...


In [63]:
loose45=loose
index=0
for i in loose45.anon_id:
    if i not in in45_loose:
        loose45=loose45.drop(index)
    index+=1
    
loose45=loose45.reset_index()
loose45

,index,answer_id,anon_id,L1,level_id,question_id,text3_len,text3 (edits made to fix word counts)
0,1,143.0,az8,Chinese,5.0,17.0,95.0,My niece is 3 years old who is my younger brot...
1,6,181.0,az8,Chinese,5.0,21.0,132.0,The Stonehenge located on the Plain of Salisbu...
2,12,542.0,az8,Chinese,5.0,62.0,188.0,Before the area of Decasia was a heartland tow...
3,15,985.0,az8,Chinese,5.0,95.0,179.0,My major was in the food science and nutrition...
4,20,1148.0,fa0,Korean,4.0,107.0,157.0,"According to increase the computer user, webah..."
...,...,...,...,...,...,...,...,...
717,1739,41656.0,bj2,Spanish,5.0,5294.0,201.0,When I had to choose my major in philosophy ma...
718,1741,42090.0,bv9,Korean,5.0,5366.0,213.0,"In the afternoon, there are a car accident and..."
719,1742,42086.0,bj2,Spanish,5.0,5366.0,242.0,"As you know, Japan has been suffering a terrib..."
720,1744,42154.0,aq1,Chinese,4.0,5385.0,359.0,Is your target to truly learn the second langu...


In [64]:
unique_t=all_three.groupby(['L1','level_id']).agg(n_uniq=('text3 (edits made to fix word counts)','nunique'))
unique_t=unique_t.reset_index()
unique_t

,L1,level_id,n_uniq
0,Chinese,3.0,11
1,Chinese,4.0,7
2,Chinese,5.0,16
3,Korean,3.0,6
4,Korean,4.0,16
5,Korean,5.0,4
6,Spanish,3.0,10
7,Spanish,4.0,10
8,Spanish,5.0,3


In [65]:
unique_t=loose34.groupby(['L1','level_id']).agg(n_uniq=('text3 (edits made to fix word counts)','nunique'))
unique_t=unique_t.reset_index()
unique_t

,L1,level_id,n_uniq
0,Chinese,3.0,30
1,Chinese,4.0,18
2,Chinese,5.0,16
3,Korean,3.0,56
4,Korean,4.0,86
5,Korean,5.0,4
6,Spanish,3.0,20
7,Spanish,4.0,21
8,Spanish,5.0,3


In [66]:
unique_t=loose45.groupby(['L1','level_id']).agg(n_uniq=('text3 (edits made to fix word counts)','nunique'))
unique_t=unique_t.reset_index()
unique_t

,L1,level_id,n_uniq
0,Chinese,3.0,11
1,Chinese,4.0,152
2,Chinese,5.0,142
3,Korean,3.0,6
4,Korean,4.0,179
5,Korean,5.0,104
6,Spanish,3.0,10
7,Spanish,4.0,66
8,Spanish,5.0,52


In [69]:
whole3=sac_lvls_rkc.query('level_id==3')

In [70]:
# function that counts the amount of tokens per language FINDS AVERAGE OF ALL LVLS IN SPECIFIED LIST
def finding_tokens(df,lang_col,text_len_col,qcounts):
    spanish=0
    korean=0
    chinese=0
    index=0
    lspanish=0
    lchinese=0
    lkorean=0
    
    for i in df['anon_id']:
        lang=df.at[index,lang_col]
        length=int(df.at[index,text_len_col])
        if lang=='Spanish':
            lspanish+=1
            spanish+=length
        elif lang=='Korean':
            lkorean+=1
            korean+=length
        elif lang=='Chinese':
            lchinese+=1
            chinese+=length
        index+=1
   # print('Spanish Token Average: ',spanish,'\nKorean Token Average: ',korean,
    #      '\nChinese Token Average:',chinese)
    print('Spanish Token Average: ',spanish/lspanish,'\nKorean Token Average: ',korean/lkorean,
          '\nChinese Token Average:',chinese/lchinese)

In [71]:
#finds the token average of the given level in the given level group (id_list) for each language
def typeAvg(df,typecol,lang, lvl):
    index=0
    #total token count
    spanish=0
    chinese=0
    korean=0
    #number of questions
    qcounts=0
    qcountc=0
    qcountk=0

    for anon in df['anon_id']:
        if df.at[index,'level_id']==lvl: # and df.at[index,'anon_id'] in id_list:
            textlen=df.at[index,typecol]
            textlen=int(textlen)
            if df.at[index,lang]=='Spanish':
                spanish+=textlen
                qcounts+=1
            elif df.at[index,lang]=='Korean':
                korean+=textlen
                qcountk+=1
            elif df.at[index,lang]=='Chinese':
                chinese+=textlen
                qcountc+=1
        index+=1
        
    print("Spanish Average: ",spanish/qcounts)
    print("Korean Average: ",korean/qcountk)
    print("Chinese Average: ",chinese/qcountc)

In [72]:
typeAvg(textcount_loose,'text3_len','L1',3)

Spanish Average:  135.85
Korean Average:  121.98979591836735
Chinese Average:  135.75


In [73]:
typeAvg(textcount_loose,'text3_len','L1',4)

Spanish Average:  213.26442307692307
Korean Average:  185.8112359550562
Chinese Average:  200.25210084033614


In [74]:
typeAvg(textcount_loose,'text3_len','L1',5)

Spanish Average:  256.6290322580645
Korean Average:  282.0131578947368
Chinese Average:  293.11764705882354


In [75]:
typeAvg(textcount_loose, 'wordtype_len', 'L1', 3)

Spanish Average:  70.38333333333334
Korean Average:  69.73469387755102
Chinese Average:  75.4423076923077


In [76]:
typeAvg(textcount_loose, 'wordtype_len', 'L1', 4)

Spanish Average:  107.72596153846153
Korean Average:  98.31460674157303
Chinese Average:  104.08683473389355


In [77]:
typeAvg(textcount_loose, 'wordtype_len', 'L1', 5)

Spanish Average:  127.54838709677419
Korean Average:  133.1798245614035
Chinese Average:  138.72268907563026


In [78]:
#finding the token/word count ratio for each language
def ratio(df,textlencol,typelencol,lang,lvl):
    index=0
    #total token count
    spanish=0
    chinese=0
    korean=0
    #unique token count
    tspanish=0
    tkorean=0
    tchinese=0
    #total question count
    qcounts=0
    qcountc=0
    qcountk=0

    for anon in df['anon_id']:
        if df.at[index,'level_id']==lvl:
            textlen=df.at[index,textlencol]
            textlen=int(textlen)
            typelen=df.at[index,typelencol]
            typelen=int(typelen)
            if df.at[index,lang]=='Spanish':
                spanish+=textlen
                tspanish+=typelen
                qcounts+=1
            elif df.at[index,lang]=='Korean':
                korean+=textlen
                tkorean+=typelen
                qcountk+=1
            elif df.at[index,lang]=='Chinese':
                chinese+=textlen
                tchinese+=typelen
                qcountc+=1
        index+=1

    #finding ratios    
    spanishavg=spanish/qcounts
    tspanishavg=tspanish/qcounts
    koreanavg=korean/qcountk
    tkoreanavg=tkorean/qcountk
    chineseavg=chinese/qcountc
    tchineseavg=tchinese/qcountc
    
    print("Spanish Ratio: ",tspanishavg/spanishavg)
    print("Korean Ratio: ",tkoreanavg/koreanavg)
    print("Chinese Ratio: ",tchineseavg/chineseavg)

In [79]:
ratio(textcount_loose,'text3_len','wordtype_len','L1',3)

Spanish Ratio:  0.5180959391485708
Korean Ratio:  0.5716436637390213
Chinese Ratio:  0.5557444397223403


In [80]:
ratio(textcount_loose,'text3_len','wordtype_len','L1',4)

Spanish Ratio:  0.5051286097522487
Korean Ratio:  0.5291101274701884
Chinese Ratio:  0.519778990068541


In [81]:
ratio(textcount_loose,'text3_len','wordtype_len','L1',5)

Spanish Ratio:  0.4970146439570109
Korean Ratio:  0.47224684676278017
Chinese Ratio:  0.4732662481006852


# Whole dataset over 3-5

In [82]:
whole=sac_lvls_rkc.query('level_id==3 or level_id==4 or level_id==5').drop(columns=['level_0'])
whole

,index,anon_id,native_language,level_id,question_id,text
0,13,ax4,Chinese,5,48,The Effects on Students Who Have a Part-time J...
1,14,ax4,Chinese,5,97,There are many different kinds of people in th...
2,15,ax4,Chinese,5,74,Should the Death Penalty Exist\nThe death pena...
3,16,ax4,Chinese,5,119,Should the Death Penalty Exist?\nThe death pen...
4,17,ax4,Chinese,5,189,The Nationality rooms built in the Cathedral o...
...,...,...,...,...,...,...
19029,46218,cz3,Korean,3,6087,I have studied Visual Design since 2002. I gra...
19030,46219,cz3,Korean,3,6119,I will describe about my ideal home. I want qu...
19031,46225,cy7,Korean,3,6066,1. emphasis\n2. appropriate\n3. requires\n4. a...
19032,46227,fq6,Chinese,3,6066,1.emphasis\n2.appropriate\n3.requires\n4.analy...


In [83]:
# all participants, not just 3-5
unique_t=whole.groupby(['native_language','level_id']).agg(n_uniq=('text','nunique'))
unique_t=unique_t.reset_index()
unique_t

,native_language,level_id,n_uniq
0,Chinese,3,1777
1,Chinese,4,2712
2,Chinese,5,3038
3,Korean,3,1466
4,Korean,4,2955
5,Korean,5,3182
6,Spanish,3,397
7,Spanish,4,697
8,Spanish,5,615


In [84]:
# this code gets how many times an id appeared (n_anon_ids) as well as an array of what levels they participated in (lvl_nums)
whole_id=whole.groupby('anon_id').agg(
    n_anon_ids_whole=('level_id', 'nunique'),
    lvl_nums_whole=('level_id', get_uniques)
)

#changes the clean_ids_lvl's index so we can access the anon_id easier
ind_whole= whole_id.reset_index()
ind_whole

,anon_id,n_anon_ids_whole,lvl_nums_whole
0,aa0,1,[5]
1,aa1,2,"[4, 3]"
2,aa3,1,[4]
3,aa5,1,[4]
4,aa8,2,"[5, 4]"
...,...,...,...
483,ha0,1,[5]
484,ha2,2,"[5, 4]"
485,ha6,2,"[3, 4]"
486,hb4,3,"[5, 4, 3]"


In [85]:
# for loop that checks if lvl_nums has all three levels in it (3,4, and 5)
# if there are three values in lvl_nums, the anon_id is appended to the all_three list
ind=0
all_three_whole=[]
append=''
for i in ind_whole.anon_id:
    if ind_whole.iat[ind, 1]==3:
        append=ind_whole.at[ind,'anon_id']
        all_three_whole.append(append)
    ind=ind+1

all_three_whole.sort()
len(all_three_whole)

47

In [89]:
whole

,index,anon_id,native_language,level_id,question_id,text
0,13,ax4,Chinese,5,48,The Effects on Students Who Have a Part-time J...
1,14,ax4,Chinese,5,97,There are many different kinds of people in th...
2,15,ax4,Chinese,5,74,Should the Death Penalty Exist\nThe death pena...
3,16,ax4,Chinese,5,119,Should the Death Penalty Exist?\nThe death pen...
4,17,ax4,Chinese,5,189,The Nationality rooms built in the Cathedral o...
...,...,...,...,...,...,...
19029,46218,cz3,Korean,3,6087,I have studied Visual Design since 2002. I gra...
19030,46219,cz3,Korean,3,6119,I will describe about my ideal home. I want qu...
19031,46225,cy7,Korean,3,6066,1. emphasis\n2. appropriate\n3. requires\n4. a...
19032,46227,fq6,Chinese,3,6066,1.emphasis\n2.appropriate\n3.requires\n4.analy...


In [91]:
all_three=whole
index=0
for i in all_three.anon_id:
    if i not in all_three_whole:
        all_three=all_three.drop(index)
    index+=1
    
#all_three.drop(columns=['level_0','index'], inplace=True)
all_three=all_three.reset_index()

In [92]:
print(all_three_whole)

['ag9', 'an5', 'aq1', 'ar8', 'as0', 'ay1', 'be7', 'bf7', 'bj2', 'bl4', 'bl7', 'bp4', 'bv9', 'bz2', 'bz5', 'cb3', 'cc4', 'cf9', 'cj8', 'cm9', 'co5', 'cv3', 'cw0', 'cz4', 'dk6', 'dp5', 'ea4', 'eq4', 'eq8', 'es9', 'ev9', 'fa2', 'fi1', 'fj4', 'fj7', 'fp5', 'fs0', 'fu6', 'fw7', 'fy1', 'gb4', 'gc2', 'gg6', 'gq8', 'gx5', 'gz2', 'hb4']


In [93]:
# checking
all_three.query('anon_id=="fi1" and level_id==3')

,level_0,index,anon_id,native_language,level_id,question_id,text
916,4319,7398,fi1,Chinese,3,32,I can't point out which restaurant is my favor...
917,4320,7399,fi1,Chinese,3,33,General Information\n\n1. Use the program for ...
918,4321,7400,fi1,Chinese,3,50,In china many farmers are hard working . A lot...
919,4322,7401,fi1,Chinese,3,99,"Otc.16.2006\nDear Mom,\n I need a new computer..."
920,4323,7402,fi1,Chinese,3,120,"Dear Mom,\n\n I need a new computer , my compu..."
921,4324,7403,fi1,Chinese,3,158,"Across Embroidery\n \n From 3 years ago , when..."
922,4325,7404,fi1,Chinese,3,187,"Across Embroidery\n From 3 yeays ago , when I ..."


In [94]:
unique_a=all_three.groupby(['native_language']).agg(n_uniq=('anon_id','nunique'))
unique_a=unique_a.reset_index()
unique_a

,native_language,n_uniq
0,Chinese,18
1,Korean,25
2,Spanish,4


In [98]:
unique_t=all_three.groupby(['native_language']).agg(n_uniq=('text','nunique'))
unique_t=unique_t.reset_index()
unique_t

,native_language,n_uniq
0,Chinese,1037
1,Korean,1769
2,Spanish,286


In [99]:
# all 3-5
unique_tl=all_three.groupby(['native_language','level_id']).agg(n_uniq=('text','nunique'))
unique_tl=unique_tl.reset_index()
unique_tl

,native_language,level_id,n_uniq
0,Chinese,3,287
1,Chinese,4,279
2,Chinese,5,472
3,Korean,3,358
4,Korean,4,607
5,Korean,5,810
6,Spanish,3,147
7,Spanish,4,89
8,Spanish,5,50


In [100]:
# all participants, not just 3-5
unique_tl=all_three.groupby(['level_id']).agg(n_uniq=('text','nunique'))
unique_tl=unique_tl.reset_index()
unique_tl

,level_id,n_uniq
0,3,777
1,4,967
2,5,1302


In [97]:
all_three

,level_0,index,anon_id,native_language,level_id,question_id,text
0,150,333,ev9,Chinese,4,2220,Dr. Schwartz's advices are very crucial and u...
1,151,334,ev9,Chinese,4,2282,There are two reasons that I think Martin is m...
2,152,335,ev9,Chinese,4,2493,"If I am going to start a new business, I will..."
3,155,338,co5,Korean,4,2220,She suggested several tips of online dating. ...
4,156,339,co5,Korean,4,2282,I am convinced that Martin really has psychic...
...,...,...,...,...,...,...,...
3340,18635,44651,bf7,Chinese,5,6040,"1. In Taiwan's middle school system, there are..."
3341,18636,44652,bf7,Chinese,5,6081,Indonesia is being threatened by tobacco indus...
3342,18655,44722,bf7,Chinese,5,6000,I am fascinated by mysteries. Mysteries are in...
3343,18656,44723,bf7,Chinese,5,6043,"No matter my present or my future life, there ..."
